In [2]:
from typing import Dict, Set, Tuple
import json
import requests
from rdflib import Graph, URIRef
from rdflib.namespace import Namespace, OWL, RDF, RDFS

## Explore the Information Retrieval Ontology

Before anything, let us shortly explore what is in the IR ontology.
The steps are:
- Load the IR ontology
- Query for the OWL classes
- Query for the OWl object properties

In [3]:
# Load the IR ontology into an RDFlib Graph
ir_onto_graph = Graph()
ir_onto_graph.parse("../information-retrieval-ontology.ttl")
ir_onto_ns = Namespace("http://www.msesboue.org/o/ir-ontology#")
ir_onto_graph.bind(prefix="ir-onto", namespace=ir_onto_ns) # setup a namespace for nicer human readable display

In [4]:
query_namespaces = { # namespaces for nicer human readable display
    "owl": OWL,
    "rdf": RDF,
    "RDFS": RDFS,
    "ir-onto": ir_onto_ns
}
query_classes = """
    SELECT ?p
    WHERE {
        ?p rdf:type owl:Class .
    }
"""
# Apply the query to the IR ontology graph and iterate through results
for r in ir_onto_graph.query(query_classes, initBindings=query_namespaces):
    print(r["p"].n3(ir_onto_graph.namespace_manager))

ir-onto:IncompatibleSearch
ir-onto:Search
ir-onto:CandidateDocument
ir-onto:Category
ir-onto:Classification
ir-onto:Document
ir-onto:EnabledCategory
ir-onto:IncompatibleDocument
ir-onto:SearchContext
ir-onto:SelectedCategory


In [5]:
query_obj_props = """
    SELECT ?p
    WHERE {
        ?p rdf:type owl:ObjectProperty .
    }
"""

# Apply the query to the graph and iterate through results
for r in ir_onto_graph.query(query_obj_props, initBindings=query_namespaces):
    print(r["p"].n3(ir_onto_graph.namespace_manager))

ir-onto:hasSearchCategory
ir-onto:categorizedBy
ir-onto:categorizes
ir-onto:enablesCategory
ir-onto:hasSubcategory
ir-onto:hasSupercategory
ir-onto:isMemberOf


## Information Retrieval Ontology usage examples

Let us now see some examples of what can we do with the IR ontology.

For these demos we will use a triple store database in a server. 
Another tutorial will explore the same examples with in memory only requirements. 
These demos require OWL inference at query time. We chose the OntoText GraphDB triple store (v10.4.0): <https://graphdb.ontotext.com/documentation/10.0/index.html>. Hence, some pieces of code might be specific to the GraphDB APIs. We will try to flag those as much as possible. Once you the docker container is running you can see their Web API documentation there: <http://localhost:7200/webapi>.
However, GraphDB implements the RDF4J REST API specification (<https://rdf4j.org/documentation/reference/rest-api/>). So we will try to make use of it as much as possible.

We use the Docker GraphDB instance without any license (the GraphDB Free version). Hence, you will need to have Docker installed and running on your computer (See Docker installation procedure: <https://docs.docker.com/get-docker/>). Let's start from there.

1. Download the GraphDB image: `docker pull ontotext/graphdb:10.4.0`
   - OntoText related documentation pointers:
     - <https://github.com/Ontotext-AD/graphdb-docker>
     - <https://hub.docker.com/r/ontotext/graphdb/>
2. Run the image: `docker run -p 127.0.0.1:7200:7200 --name graphdb-ir-onto -t ontotext/graphdb:10.4.0`
3. From now on the rest is in the code.

In [7]:
# Setting up the repository we will work with

# Load the repository configuration (WARNING: specific to GraphDB)
repo_config_graph = Graph()
repo_config_graph.parse("./data/ir-onto-demo-graphdb-config.ttl")
repo_config_ttl_string = repo_config_graph.serialize(format="turtle")

headers = {
    "Accept": "application/json",
}
data = {
    "config": ("config.ttl", repo_config_ttl_string)
}

# Uses the GraphDB REST API (WARNING: specific to GraphDB)
r = requests.post("http://localhost:7200/rest/repositories", headers=headers, files=data)
print(r.status_code)
print(r.text)

201



In [8]:
# Check that the repository is created
r = requests.get('http://localhost:7200/repositories', headers={"Content-type": "application/rdf+xml"})
print(r.status_code)
print(r.text)

200
uri,id,title,readable,writable
http://localhost:7200/repositories/ir-onto-demo,ir-onto-demo,,true,true



### Some utility functions

In [9]:
from typing import Optional


DB_IP = "localhost"
DB_PORT = "7200"
DB_URL = f"http://{DB_IP}:{DB_PORT}"
REPOSITORY_ID = "ir-onto-demo"

def sparql_select(sparql_query: str, use_inference: bool=False) -> Dict:
    
    query_resp = requests.get(
        f"{DB_URL}/repositories/{REPOSITORY_ID}",
        headers={"Accept": "application/sparql-results+json"},
        params={
            "query": sparql_query,
            "infer": use_inference
        }
    )

    return query_resp

def show_owl_classes(namespace: Optional[Tuple[str]]=None) -> None:

    query_classes = """
            SELECT ?p
            WHERE {
                ?p rdf:type owl:Class .
            }
        """
    resp = sparql_select(sparql_query=query_classes)
    json_resp = json.loads(resp.text)
    class_uris = [binding["p"]["value"] for binding in json_resp["results"]["bindings"]]

    if (namespace is not None) and (len(namespace)==2):
        class_uris = [uri.replace(namespace[1], namespace[0] + ":") for uri in class_uris]

    for uri in class_uris:
        print(uri)

def show_owl_obj_props(namespace: Optional[Tuple[str]]=None) -> None:

    query_classes = """
            SELECT ?p
            WHERE {
                ?p rdf:type owl:ObjectProperty .
            }
        """
    resp = sparql_select(sparql_query=query_classes)
    json_resp = json.loads(resp.text)
    class_uris = [binding["p"]["value"] for binding in json_resp["results"]["bindings"]]

    if (namespace is not None) and (len(namespace)==2):
        class_uris = [uri.replace(namespace[1], namespace[0] + ":") for uri in class_uris]

    for uri in class_uris:
        print(uri)

def sparql_insert(sparql_query: str) -> None:
    
    query_resp = requests.post(
        f"{DB_URL}/repositories/{REPOSITORY_ID}/statements",
        headers={"Content-type": "application/sparql-update"},
        data=sparql_query
    )

    if not query_resp.status_code == 204: # check that the triples have been created
        print(f"Something went wrong. HTTP response Code: {query_resp.status_code}")
        print(f"HTTP response body: {query_resp.text}")
    else:
        print("OK")

In [10]:
# Let us now load the IR ontology

# RDF4J does not let you simply add an RDF file to a repository without 
# erasing what was already in it.
# Hence we need to use a SPARQL INSERT query

ir_onto_ttl_string = ir_onto_graph.serialize(format="turtle")
sparql_query = f"""
INSERT DATA {{
    {ir_onto_ttl_string}
}}
"""
sparql_insert(sparql_query)

OK


In [11]:
ir_onto_ns = ("ir-onto", "http://www.msesboue.org/o/ir-ontology#")
show_owl_classes(namespace=ir_onto_ns)

ir-onto:IncompatibleDocument
ir-onto:Document
ir-onto:IncompatibleSearch
ir-onto:Search
ir-onto:SearchContext
ir-onto:SelectedCategory
ir-onto:Category
ir-onto:Classification
ir-onto:EnabledCategory
ir-onto:CandidateDocument


In [12]:
ir_onto_ns = ("ir-onto", "http://www.msesboue.org/o/ir-ontology#")
show_owl_obj_props(namespace=ir_onto_ns)

ir-onto:hasSearchCategory
ir-onto:categorizedBy
ir-onto:categorizes
ir-onto:hasSubcategory
ir-onto:hasSupercategory
ir-onto:enablesCategory
ir-onto:isMemberOf


## Insert some example data

For our examples we will use a version of the well-known Pizza ontology. We will use the one from this repository: <https://github.com/owlcs/pizza-ontology/>

In [13]:
pizza_onto_graph = Graph()
pizza_onto_graph.parse("C:/Users/msesboue/OneDrive - TRACEPARTS/These_RESPONDING/ontologies/external_ontologies/pizza.owl")
pizza_onto_ns = Namespace("http://www.co-ode.org/ontologies/pizza/pizza.owl#")
pizza_onto_graph.bind(prefix="pizza-onto", namespace=pizza_onto_ns)

In [14]:
pizza_taxos_graph = Graph()
pizza_taxos_graph.parse("./data/pizza-taxonomies.ttl")
pizza_taxos_ns = Namespace("http://www.msesboue.org/o/ir-onto-pizza-demo#")
pizza_taxos_graph.bind(prefix="pizza-taxos", namespace=pizza_taxos_ns)

In [16]:
query_namespaces = { # namespaces for nicer human readable display
    "owl": OWL,
    "rdf": RDF,
    "RDFS": RDFS,
    "ir-onto": ir_onto_ns
}

print("Classes")
# Apply the query to the IR ontology graph and iterate through results
for r in pizza_taxos_graph.query(query_classes, initBindings=query_namespaces):
    print(r["p"].n3(pizza_taxos_graph.namespace_manager))
print()
print("Object Properties")
for r in pizza_taxos_graph.query(query_obj_props, initBindings=query_namespaces):
    print(r["p"].n3(pizza_taxos_graph.namespace_manager))

Classes
pizza-taxos:Country
pizza-taxos:Pizza
pizza-taxos:PizzaBase
pizza-taxos:PizzaKind
pizza-taxos:PizzaTopping
pizza-taxos:Spiciness

Object Properties
pizza-taxos:hasBase
pizza-taxos:hasCountryOfOrigin
pizza-taxos:hasIngredient
pizza-taxos:hasPizzaKind
pizza-taxos:hasSpiciness
pizza-taxos:hasTopping
pizza-taxos:makesIt
ir-onto:hasSupercategory


In [17]:
# Lets add the pizza taxonomies to our DB
pizza_taxos_ttl_string = pizza_taxos_graph.serialize(format="turtle")
sparql_query = f"""
INSERT DATA {{
    {pizza_taxos_ttl_string}
}}
"""
sparql_insert(sparql_query)

OK


In [18]:
print("Classes")
show_owl_classes()
print()
print("Object Properties")
show_owl_obj_props()

Classes
http://www.msesboue.org/o/ir-ontology#IncompatibleDocument
http://www.msesboue.org/o/ir-ontology#Document
http://www.msesboue.org/o/ir-ontology#IncompatibleSearch
http://www.msesboue.org/o/ir-ontology#Search
http://www.msesboue.org/o/ir-ontology#SearchContext
http://www.msesboue.org/o/ir-ontology#SelectedCategory
http://www.msesboue.org/o/ir-ontology#Category
http://www.msesboue.org/o/ir-ontology#Classification
http://www.msesboue.org/o/ir-ontology#EnabledCategory
http://www.msesboue.org/o/ir-ontology#CandidateDocument
http://www.msesboue.org/o/ir-onto-pizza-demo#Pizza
http://www.msesboue.org/o/ir-onto-pizza-demo#PizzaBase
http://www.msesboue.org/o/ir-onto-pizza-demo#Country
http://www.msesboue.org/o/ir-onto-pizza-demo#Spiciness
http://www.msesboue.org/o/ir-onto-pizza-demo#PizzaTopping
http://www.msesboue.org/o/ir-onto-pizza-demo#PizzaKind

Object Properties
http://www.msesboue.org/o/ir-ontology#hasSearchCategory
http://www.msesboue.org/o/ir-ontology#categorizedBy
http://www.ms

At the moment our triple store contains both the IR ontology graph and the pizza taxonomies graph. However, they are not yet linked.

To link both graph (our domain and data graph) in a meaningful manner, we need to add some triples to define:

- What are the categories in our pizza taxonomies graph?
- What are the documents in our pizza taxonomies graph?
- What are the relations (i.e., object properties) that should enable other categories when the their subject category is selected?
- What are the relations used to categorise our documents?

These triples will form our mapping between the data and domain graph and will enable reasoning over our data. 

In [15]:
mapping_graph = Graph()

mapping_graph.add((ir_onto_ns.hasSubcategory, RDFS.subPropertyOf, ir_onto_ns.enablesCategory))
mapping_graph.add((ir_onto_ns.hasSubcategory, RDF.type, OWL.SymmetricProperty))
mapping_graph.add((pizza_taxos_ns.makesIt, RDFS.subPropertyOf, ir_onto_ns.enablesCategory))

mapping_graph.add((pizza_taxos_ns.hasTopping, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))
mapping_graph.add((pizza_taxos_ns.hasIngredient, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))
mapping_graph.add((pizza_taxos_ns.hasCountryOfOrigin, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))
mapping_graph.add((pizza_taxos_ns.hasPizzaKind, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))
mapping_graph.add((pizza_taxos_ns.hasSpiciness, RDFS.subPropertyOf, ir_onto_ns.categorizedBy))

<Graph identifier=N71ab5c4143744d239739042d6c89055c (<class 'rdflib.graph.Graph'>)>

## Our first user search

In [27]:
def query_insert_search_context(categories: Set[str], instance_ns_uri: str) -> str:
    # the categories should be the category instances name (without the namespace)

    category_triples = "".join([f", pizza:{cate}" for cate in categories])
    query = f"""
        PREFIX ir-onto: <http://www.msesboue.org/o/ir-ontology#>
        PREFIX pizza: <{instance_ns_uri}>
        INSERT DATA {{
            pizza:user_search a ir-onto:SearchContext ;
                    ir-onto:hasContextCategory {category_triples[2:]} .
        }}
    """

    return query

def query_delete_search_context(categories: Set[str], instance_ns_uri: str) -> str:
    # the categories should be the category instances name (without the namespace)

    category_triples = "".join([f", pizza:{cate}" for cate in categories])
    query = f"""
        PREFIX ir-onto: <http://www.msesboue.org/o/ir-ontology#>
        PREFIX pizza: <{instance_ns_uri}>
        DELETE DATA {{
            pizza:user_search a ir-onto:SearchContext ;
                    ir-onto:hasContextCategory {category_triples[2:]} .
        }}
    """

    return query

In [28]:
query_insert_search = query_insert_search_context(categories=["MeatTopping", "PizzaBase"], instance_ns_uri="http://www.msesboue.org/o/ir-onto-pizza-demo#")
print(query_insert_search)
query_delete_search = query_delete_search_context(categories=["MeatTopping", "PizzaBase"], instance_ns_uri="http://www.msesboue.org/o/ir-onto-pizza-demo#")


        PREFIX ir-onto: <http://www.msesboue.org/o/ir-ontology#>
        PREFIX pizza: <http://www.msesboue.org/o/ir-onto-pizza-demo#>
        INSERT DATA {
            pizza:user_search a ir-onto:SearchContext ;
                    ir-onto:hasContextCategory pizza:MeatTopping, pizza:PizzaBase .
        }
    


In [22]:
r = requests.post(
    'http://localhost:7200/repositories/ir-onto-demo/statements', 
    data=query_insert_search, 
    headers={"Content-type": "application/sparql-update"}
)
print(r.status_code)
print(r.text)

if r.status_code == 204: # check that the triples have been created
    r = requests.get(
        'http://localhost:7200/repositories/ir-onto-demo/size'
        )
    print(r.text)

204

459


In [24]:
query_enabled_categories = """
    PREFIX ir-onto: <http://www.msesboue.org/o/ir-ontology#>
    SELECT ?cate
    WHERE {
        ?cate rdf:type ir-onto:EnabledCategory .
    }
"""

sparql.setQuery(query_enabled_categories)
query_res = sparql.queryAndConvert()

for r in query_res["results"]["bindings"]:
    print(r)

{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_hamTopping'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_chickenTopping'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_deepPanBase'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_hotSpicedBeefTopping'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_peperoniSausageTopping'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_thinAndCrispyBase'}}


In [25]:
q_make_enables_transitive = """
    PREFIX ir-onto: <http://www.msesboue.org/o/ir-ontology#>
    INSERT DATA {
        ir-onto:enablesCategory rdf:type  owl:TransitiveProperty.
    }
"""

r = requests.post(
    'http://localhost:7200/repositories/ir-onto-demo/statements', 
    data=q_make_enables_transitive, 
    headers={"Content-type": "application/sparql-update"}
)
print(r.status_code)
print(r.text)

if r.status_code == 204: # check that the triples have been created
    r = requests.get(
        'http://localhost:7200/repositories/ir-onto-demo/size'
        )
    print(r.text)

204

460


In [26]:
query_enabled_categories = """
    PREFIX ir-onto: <http://www.msesboue.org/o/ir-ontology#>
    SELECT ?cate
    WHERE {
        ?cate rdf:type ir-onto:EnabledCategory .
    }
"""

sparql.setQuery(query_enabled_categories)
query_res = sparql.queryAndConvert()

for r in query_res["results"]["bindings"]:
    print(r)

{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#ParmaHamTopping'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_hamTopping'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_chickenTopping'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_deepPanBase'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_hotSpicedBeefTopping'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_peperoniSausageTopping'}}
{'cate': {'type': 'uri', 'value': 'http://www.msesboue.org/o/ir-onto-pizza-demo#_thinAndCrispyBase'}}


In [30]:
print(query_delete_search)


        PREFIX ir-onto: <http://www.msesboue.org/o/ir-ontology#>
        PREFIX pizza: <http://www.msesboue.org/o/ir-onto-pizza-demo#>
        DELETE DATA {
            pizza:user_search a ir-onto:SearchContext ;
                    ir-onto:hasContextCategory pizza:MeatTopping, pizza:PizzaBase .
        }
    


In [31]:
r = requests.post(
    'http://localhost:7200/repositories/ir-onto-demo/statements', 
    data=query_delete_search, 
    headers={"Content-type": "application/sparql-update"}
)
print(r.status_code)
print(r.text)

if r.status_code == 204: # check that the triples have been created
    r = requests.get(
        'http://localhost:7200/repositories/ir-onto-demo/size'
        )
    print(r.text)

204

457


In [32]:
sparql.setQuery(query_enabled_categories)
query_res = sparql.queryAndConvert()

for r in query_res["results"]["bindings"]:
    print(r)